In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from sklearn.metrics import classification_report, confusion_matrix
from torchvision import transforms
from torch import nn, optim

2024-09-04 20:57:11.926769: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-04 20:57:11.941373: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-04 20:57:11.945451: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-04 20:57:11.955925: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-04 20:57:12.873932: W tensorflow/compiler/tf2

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [3]:
# Data augmentation
augmentation = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ])
}

In [4]:
# We can tune this
input_class     = 3
batch_size      = 32
img_height      = 224
img_width       = 224
learning_rate   = 0.4
max_epochs      = 30

In [5]:
# Data augmentation
augmentation = {
    "train": transforms.Compose(
        transforms=[
            transforms.Resize((224, 224)),
            transforms.RandomRotation(degrees=30),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
        ]
    )
}

In [6]:
class Train:
    def __init__(self) -> None:
        (train_ds, val_ds) = tf.keras.preprocessing.image_dataset_from_directory(
            "datasets/Gastrovision",
            validation_split=0.15,
            subset="both",
            seed=69420,
            image_size=(img_height, img_width),
            batch_size=batch_size,
            shuffle=True
        )
        
    def train_net(self):
        model = torchvision.models.resnet50(weights=False).to(device)
        
        for param in model.parameters():
            param.requires_grad=True
            
        n_inputs = model.fc.in_features
        model.fc = nn.Sequential(
            nn.Linear(in_features=n_inputs, out_features=256),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(256, input_class),
            nn.LogSoftmax(dim=1)
        )
        
        model.to(device=device)
        optimizer = optim.Adam(params=model.parameters(), lr=learning_rate, weight_decay=1e-4)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode="min", patience=4, verbose=True)
        criterion = nn.NLLLoss()
        val_f1_max = 0.0
        epochs  = []
        lossesT = []
        lossesV = []
        
        for epoch in range(max_epochs):
            print(f"Epoch {epoch + 1}/{max_epochs}")
            


In [7]:
(train_ds, val_ds) = tf.keras.preprocessing.image_dataset_from_directory(
    "datasets/Gastrovision",
    validation_split=0.15,
    subset="both",
    seed=69420,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 2426 files belonging to 3 classes.
Using 2063 files for training.
Using 363 files for validation.


I0000 00:00:1725458246.491495    7060 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-04 20:57:26.496076: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [8]:
# ResNet
resnet_base = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)
resnet_base.trainable=True

for layer in resnet_base.layers[:-4]:
    layer.trainable=False

In [9]:
inputs = keras.Input(shape=(224, 224, 3))
x = keras.applications.resnet50.preprocess_input(inputs)
x = resnet_base(x)
x = Flatten()(x)
x = Dense(256)(x)
x = Dropout(0.5)(x)
outputs = Dense(units=8, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [16]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_3          │ (None, 224, 224)  │          0 │ input_layer_5[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_4          │ (None, 224, 224)  │          0 │ input_layer_5[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_5          │ (None, 224, 224)  │          0 │ input_layer_5[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack_1 (Stack)     │ (None, 224, 224,  │          0 │ get_item_3[0][0], │
│                     │ 3)                │            │ get_item_4[0][0], │
│                     │                   │            │ get_item_5[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 224, 224,  │          0 │ stack_1[0][0]     │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50            │ (None, 7, 7,      │ 23,587,712 │ add_1[0][0]       │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 100352)    │          0 │ resnet50[1][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │ 25,690,368 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 8)         │      2,056 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 49,280,136 (187.99 MB)

 Trainable params: 26,747,144 (102.03 MB)

 Non-trainable params: 22,532,992 (85.96 MB)

In [10]:
model.compile(
    optimizer="adam",
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

In [11]:
def get_early_stopping():
    return tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

early_stopping = get_early_stopping()

callbacks = [early_stopping]

In [12]:
history = model.fit(train_ds, validation_data=val_ds, batch_size=batch_size, epochs=30, callbacks=callbacks)

Epoch 1/30
10/65 ━━━━━━━━━━━━━━━━━━━━ 1:16 1s/step - accuracy: 0.4575 - loss: 64.6517

KeyboardInterrupt: 